# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from time import time
import datetime

import torch
from tqdm.notebook import tqdm

import json
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from model import FESD, FESDv2, train, val, test

from utils.mode import Mode
from utils import get_model_iter, get_model_iter_all

from data import Frame


num_gpus = torch.cuda.device_count()
is_cuda = torch.cuda.is_available()
print(f"Num cuda GPUs: {num_gpus}")

Num cuda GPUs: 0


## Model

Build the model according to the chosen mode

### Train Model

In the following we define the training function and train a network on the training data.

In [2]:
(780 / 26) * len(['E-0.00', 'E-0.01', 'E-1.00', 'E-1.02', 'E-2.00', 'E-2.03', 'E-3.00', 'E-3.02'])

240.0

In [4]:
batchsize = 32
im_size = 300
epochs = 50
# gradient clipping margin
clip = 0.5
label_smoothing = 0.0
learning_rate = 0.0001

test_exercises = ['E-0.00', 'E-0.01', 'E-1.00', 'E-1.02', 'E-2.00', 'E-2.03', 'E-3.00', 'E-3.02']

all_modes = True
if not all_modes:
    mode = Mode.FULL_BODY

model_columns = ["epoch", "iteration", "joint_id",
                  "gts", "preds", "confidences", 
                  "Avg loss", "loss", "accuracy", 
                  "tp", "tn", "fp", "fn", "precision", "recall", "f1", 
                  "cohens_kappa", "learning_rate",
                  "train_test", "exercise", "simplified", "mode", "use_v2"]
                  
for use_v2 in [False, True]:
  model_id = f"{'v2' if use_v2 else 'v1'}{'' if all_modes else '_' + mode.to_str()}_bs_{batchsize}_is_{im_size}_e_{epochs}_ls_{label_smoothing}_lr_{learning_rate}"

  CE = torch.nn.CrossEntropyLoss(label_smoothing=label_smoothing)
  if is_cuda:
      CE = CE.cuda()

  model_iterator = []
  if all_modes:
      model_iterator = get_model_iter_all(is_cuda, use_v2, test_exercises, epochs, batchsize, im_size, clip, learning_rate)
  else:
      model_iterator = get_model_iter(mode, is_cuda, use_v2, test_exercises, epochs, batchsize, im_size, clip, learning_rate)
                    
  df_model = pd.DataFrame(columns=model_columns)
  pb = tqdm(range(1, epochs + 1), desc='Epoch')

  model_result_dir = f"./results/{model_id}"
  dir_fallback = "_are_you_kidding_me_with_this_many_models"

  try:
    Path(model_result_dir).mkdir(parents=True)
  except:
    for i in dir_fallback:
      model_result_dir += f"{i}"
      try:
        Path(model_result_dir).mkdir(parents=True)
        break
      except:
        pass

  model_result_dir = Path(model_result_dir)

  for epoch in pb:
      print(f"--- {epoch:3d} ---")
      for mode, model, optimizer, scheduler, train_loader, test_loader in model_iterator:
          tic = time()
          torch.cuda.empty_cache()
          
          train(train_loader, model, optimizer, CE, scheduler, clip, epoch, is_cuda, mode, df_model, use_v2)
          test(test_loader, model, CE, epoch, is_cuda, mode, df_model, use_v2)

          crit_1 = df_model["epoch"] == epoch
          crit_2 = df_model["mode"] == mode.name.lower()
          crit_3 = df_model["train_test"] == "test"
          last_row = df_model[crit_1 & crit_2 & crit_3].mean(numeric_only=True)
          last_row["p"] = last_row["tp"] + last_row["fp"]
          last_row["n"] = last_row["tn"] + last_row["fn"]
          tp = df_model[crit_1 & crit_2 & crit_3]["tp"].sum()
          tn = df_model[crit_1 & crit_2 & crit_3]["tn"].sum()
          fp = df_model[crit_1 & crit_2 & crit_3]["fp"].sum()
          fn = df_model[crit_1 & crit_2 & crit_3]["fn"].sum()
          kappa = (2 * (tp * tn) - (fn * fp)) / ((tp + fp) * (fp + tn) + (tp + fn) * (fn + tn))
          df_model[crit_1 & crit_2 & crit_3]["cohens_kappa"] = kappa
          last_row["positives"] = last_row["p"] / (last_row["n"] + last_row["p"])
          pb.set_description(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.5f}, loss: {last_row["Avg loss"]:.3f})')
          
          print(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.3f}, loss: {last_row["Avg loss"]:.5f}, acc: {last_row["accuracy"]:.3f}, f1: {last_row["f1"]:.3f}, precision: {last_row["precision"]:.3f}, recall: {last_row["recall"]:.3f}, kappa: {kappa:.3f}, p/(p + n): {last_row["positives"]:.3f}, time: {time() - tic:.2f}s)')        

          if (epoch) % 10 == 0:
              torch.save(model.state_dict(), os.path.join(Path('checkpoints'), f"{model_id}_{mode.name.lower()}_{epoch}_ckpt.pth")) 
      
  for mode, model, _, _, _, _ in model_iterator:
      torch.save(model.state_dict(), model_result_dir / f"{mode.name.lower()}_last_ckpt.pth") 
      print(f"model saved {os.path.join(model_result_dir, f'last_ckpt.pth')}!")

  df_model.to_parquet(model_result_dir / f'ModelAnalysis.parquet.gzip', compression='gzip') 

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

---   1 ---
Epoch (mode: body parts, lr: 0.001, loss: 0.38594, acc: 0.873, f1: 0.873, precision: 0.873, recall: 0.873, kappa: 0.000, p/(p + n): 1.000, time: 403.26s)


KeyboardInterrupt: 